# V2
I split the process into 2 steps.   
The first one try to fix the error, such as typo, and remove reduntent newlines ('\n').  
And the second one focus on translation.  

But the removing "\n" seems not works

## Gemini Pro API and OpenAI API
Check [Bilingual-NovelTranslator-v1.ipynb](Bilingual-NovelTranslator-v1.ipynb) for more information.

**Note** : Remember to fill you api key in `.env` file

In [1]:
%load_ext autoreload
%autoreload 2
    
import utils

geminipro, openai = utils.init_llm()

In [2]:
def split_paragraph(source_path):
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1024,
        chunk_overlap=0,
        separators= ["\n\n\n\n","\n\n\n","\n\n"]
        # length_function=len,
        # is_separator_regex=True,
    )
    content=""
    with open(source_path, 'r') as f:
        content = f.read()
        # aaa = content.split("Contents",2)[1]
    
        
    return text_splitter.split_text(content)

In [3]:
def fix_paragraph(llm, paragraphs, target_file=None):
    from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
    from langchain_core.output_parsers.string import StrOutputParser
    from langchain.callbacks.tracers import ConsoleCallbackHandler
    prompt = PromptTemplate.from_template(
    """
    You are an article assistant.
    Your task is to check and fix only misspelling and typo. and redundent linebreak in this article.
    You should remove any linebreak in a complete sentence.
    Do not add any other comment or change the words.
    
    Here's the article we are going to check:
    
    {input}
    
    """)
    
    
    chain = prompt | llm | StrOutputParser()
    
    fixed_paragraphs = []
    from tqdm.notebook import tqdm, trange
    with tqdm(total=len(paragraphs)) as progress_bar:
        for i, paragraph in enumerate(paragraphs):
            if len(paragraph.strip()) > 0:
                temp = ""
                temp = chain.invoke({"input": paragraph})
                # for chunk in chain.stream({"input": paragraph}):
                #     temp += chunk
                #     print(chunk, end="", flush=True)
                # print("\n------\n")
                fixed_paragraphs.append(temp)
                if target_file:
                    with open(target_file, 'a+') as f:
                            f.write(temp)
                            f.write('\n')
            progress_bar.update(1)
    return fixed_paragraphs

In [4]:
def translate_paragraph(llm, paragraphs, target_file=None):
    from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
    from langchain_core.output_parsers.string import StrOutputParser
    prompt = PromptTemplate.from_template(
        """You are a professional translation assistant. Your task is to translate a paragraph from english to traditional Chinese.
      
You should first sepatate each paragraph into several complete sentences.
Each sentence should be prepended with a newline(\n).
Then you should translate each sentence into zh-tw followed by the original sentence, separated by a newline(\n).
Add one more newline(\n) at end before processing next sentence.
Make sure no original text or translations should be skipped.

Example:
---
input:
Before one girl and another even younger one stood a figure in full plate armor brandishing a sword.
The blade swung, sparkling in the sunlight as if to say that taking their lives in a single stroke would be an act of mercy.
---
output:
在一名少女以及比她更年輕的少女面前，站著一位身穿全身板甲、揮舞著劍的男子。
Before one girl and another even younger one stood a figure in full plate armor brandishing a sword.

刀鋒揮動，在陽光下閃爍，彷彿在說一刀奪命是仁慈的作為。
The blade swung, sparkling in the sunlight as if to say that taking their lives in a single stroke would be an act of mercy.


---

The original paragraph is as following:

{input}

""")
    
    chain = prompt | llm | StrOutputParser()
    translated=[]
    from tqdm.notebook import tqdm, trange
    with tqdm(total=len(paragraphs)) as progress_bar:
        for i, paragraph in enumerate(paragraphs):
            if len(paragraph.strip()) > 0:
                temp = ""
                for chunk in chain.stream({"input": paragraph}):
                    print(chunk, end="", flush=True)
                    temp += chunk
                print("\n------\n")    
                translated.append(temp)
                if target_file:
                    with open(target_file, 'a+') as f:
                        f.write(temp)
                        f.write('\n')
            progress_bar.update(1)

In [5]:
source_filenames = ["alice_in_wonderland.txt"]
for i, source_file in enumerate(source_filenames):
    print(source_file)
    path = source_file
    paragraphs = split_paragraph(path)[1:5]
    fixed_paragraphs = fix_paragraph(geminipro, paragraphs)
    translations = translate_paragraph(geminipro, fixed_paragraphs)

alice_in_wonderland.txt


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

章節一
Down the Rabbit-Hole
------

愛麗絲開始對坐在河岸邊的姊姊感到很厭煩，而且無所事事：她曾一兩次偷看姊姊正在讀的書，但書裡沒有圖片或對話，愛麗絲心想：「一本沒有圖片或對話的書有什麼用？」
Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, `and what is the use of a book,' thought Alice `without pictures or conversation?'


於是她開始思考（儘管天氣炎熱讓她感到昏昏欲睡又遲鈍，但她還是盡力思考），製作雛菊花環的樂趣是否值得起身去摘雛菊，就在這時，一隻有著粉紅眼睛的白兔跑近她身邊。
So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her.
------

兔子從背心口袋裡掏出一只懷錶，看了看，然後匆匆忙忙地跑開了。
There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, `Oh dear!  Oh dear!  I shall be late!' (when s

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


兔子洞直直地延伸出去，像條隧道一樣，然後突然向下傾斜，如此突然以致愛麗絲連思考要如何停止自己都沒有時間，就發現自己正在一個非常深的井裡往下墜落。

The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well.

井不是非常深，就是她墜落的速度非常慢，因為她在往下墜的過程中，有充裕的時間四處觀望，並好奇接下來會發生什麼事。首先，她試著往下看，找出自己會掉到哪裡，但實在太暗了，什麼都看不見；接著她看向井壁，注意到井壁上佈滿了櫥櫃和書架；這裡那裡，她看到有地圖和圖片掛在掛鉤上。她在經過其中一個架子時，拿下來一個罐子；罐子上標示著「橘子果醬」，但讓她非常失望的是，罐子是空的：她不喜歡把罐子丟掉，害怕會砸到人，因此設法在經過時，把它放進其中一個櫥櫃裡。

Either the well was very deep, or she fell very slowly, for she had plenty of time as she went down to look about her and to wonder what was going to happen next. First, she tried to look down and make out what she was coming to, but it was too dark to see anything; then she looked at the sides of the well, and noticed that they were filled with cupboards and book-shelves; here and there she saw maps and pictures hung upon pegs. She took down a jar from one of the shelves as she 